In [48]:
# Import packages
 # import fuzzywizzy?
import pandas as pd
import numpy as np
import pymongo
import string
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.util import ngrams
from string import punctuation
from gensim.models import KeyedVectors
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers.merge import concatenate
from keras.models import Model
from keras.layers.normalization import BatchNormalization
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [3]:
# open up a Mongo Connection and create a cursor
myclient = pymongo.MongoClient("mongodb://127.0.0.1:27017/")
curlydb = myclient['Naturaly_Curly_db']
curly_collection = curlydb['curly_profiles']

In [4]:
# Access all the unique items and store them to a list
query_1 = curly_collection.find({})
test = []
for x in query_1:
    test.append(x['signature'])

# This is the number of total entire in the database
print(len(test)) 

532489


In [11]:
# Create a set of unique items in the db and inspect it
unique_sigs = list(set(test))
print(type(unique_sigs))
print(len(unique_sigs))
print(unique_sigs[5000:5003])

<class 'list'>
7961
["3b curls, big chop Nov 07\n\nCurrent fave products: Aubrey Organics Honeysuckle Rose Shampoo and Conditioner, Aveda Curl Enhancer, Aveda Flaxseed Aloe gel\n\nStill like: Ouidad CC gel\nDon't like: Deva ArcAngell", '2b waves\nMedium texture, high porosity\nBronde & shoulder length, trying to grow it past my shoulder blades.\nWent natural on June 4th, 2010\nMy hair is hungry for PROTEIN!! :munky2:', '-3A/Medium/Low porosity\n\n-Cg since 01/2012\n-Growing out since 11/2011\n\n-Co-wash: Suave Naturals']


In [35]:
# test a cleaning function on a small sample size
samples = unique_sigs[0:50]
samples

['',
 'Lions, Tigers and My Blog',
 "Very modified CG. Activate and B2B pomegranate Condish. Re:coil, Joiwhip, LOOB, HE set me up and orange bottle. Boots, cant' remember anymore.\nIn a place where dry and burned straight hair is so common my wild spanish curls are my greatest weapon!",
 'Mod CG since 06/08/2011-trying to figure it all out!\n\nHair type: 2b/2c, medium texture, high (??) porosity\nCo-Wash: Tresemme Naturals\nRO: AOHR\nLI: Tresemme Naturals\nStylers: Jessicurl RR + CC, AG Re:Coil (still trying to love this), and Curl Keeper\nGels: EcoStyler, BRHG, Got2B Spiked Up\nTried: Aloe (ok for frizz), FSG (so wanted this to work)',
 '3a/b.',
 "\nFine, curly/wavy, prone to breakage. Still figuring out curl pattern...3A on some days (only w/ product), 2C on others (but not coarse). Would love help on hair-typing! Low porosity (I think). I thought it was low-density, but a hairstylist told me I have tiny hairs growing in underneath that break, so that's why it's thin). Product junkie

In [39]:
lower_sample=[]
for sample in samples:
    lower_sample.append(sample.lower().split())
lower_sample


[[],
 ['lions,', 'tigers', 'and', 'my', 'blog'],
 ['very',
  'modified',
  'cg.',
  'activate',
  'and',
  'b2b',
  'pomegranate',
  'condish.',
  're:coil,',
  'joiwhip,',
  'loob,',
  'he',
  'set',
  'me',
  'up',
  'and',
  'orange',
  'bottle.',
  'boots,',
  "cant'",
  'remember',
  'anymore.',
  'in',
  'a',
  'place',
  'where',
  'dry',
  'and',
  'burned',
  'straight',
  'hair',
  'is',
  'so',
  'common',
  'my',
  'wild',
  'spanish',
  'curls',
  'are',
  'my',
  'greatest',
  'weapon!'],
 ['mod',
  'cg',
  'since',
  '06/08/2011-trying',
  'to',
  'figure',
  'it',
  'all',
  'out!',
  'hair',
  'type:',
  '2b/2c,',
  'medium',
  'texture,',
  'high',
  '(??)',
  'porosity',
  'co-wash:',
  'tresemme',
  'naturals',
  'ro:',
  'aohr',
  'li:',
  'tresemme',
  'naturals',
  'stylers:',
  'jessicurl',
  'rr',
  '+',
  'cc,',
  'ag',
  're:coil',
  '(still',
  'trying',
  'to',
  'love',
  'this),',
  'and',
  'curl',
  'keeper',
  'gels:',
  'ecostyler,',
  'brhg,',
  'got

In [43]:
tokens = word_tokenize(samples[33])
tokens

['2b/3a',
 ',',
 'primarily',
 'use',
 'Jessicurl',
 'and',
 'DevaCurl',
 'products',
 ',',
 'Curls',
 'Hair',
 'Tea',
 'conditioner',
 'and',
 'various',
 'gels',
 '.',
 'I',
 "'m",
 'modified',
 'CG-ish',
 'since',
 '5/04',
 '.']

In [49]:
def generate_ngrams(s, n):
    # Convert to lowercases
    s = s.lower()
    
    # Replace all none alphanumeric characters with spaces
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    
    # Break sentence in the token, remove empty tokens
    tokens = [token for token in s.split(" ") if token != ""]
    
    # Use the zip function to help us generate n-grams
    # Concatentate the tokens into ngrams and return
    ngrams = zip(*[tokens[i:] for i in range(n)])
    return [" ".join(ngram) for ngram in ngrams]

In [50]:
generate_ngrams(samples[33], 2)

['2b 3a',
 '3a primarily',
 'primarily use',
 'use jessicurl',
 'jessicurl and',
 'and devacurl',
 'devacurl products',
 'products curls',
 'curls hair',
 'hair tea',
 'tea conditioner',
 'conditioner and',
 'and various',
 'various gels',
 'gels i',
 'i m',
 'm modified',
 'modified cg',
 'cg ish',
 'ish since',
 'since 5',
 '5 04']

In [53]:
s = re.sub(r'[^a-zA-Z0-9\s]', ' ', samples[33])
tokens = word_tokenize(s)
vocabulary = set(tokens)
print(len(vocabulary))
frequency_dist = nltk.FreqDist(tokens)
sorted(frequency_dist,key=frequency_dist.__getitem__, reverse=True)[0:50]


22


['and',
 '2b',
 '3a',
 'primarily',
 'use',
 'Jessicurl',
 'DevaCurl',
 'products',
 'Curls',
 'Hair',
 'Tea',
 'conditioner',
 'various',
 'gels',
 'I',
 'm',
 'modified',
 'CG',
 'ish',
 'since',
 '5',
 '04']

In [66]:
s = re.sub(r'[^a-zA-Z0-9\s]', ' ', samples[34])
s = s.lower()
s

'mostly 3a  some 3b 2c fine med texture med porosity low density\n\nwash  alternate co washing with v05   soap bars\nrinse out  rotate tj s n s  aohr  lvpng  gvp k pak  cj ca daily condish\nacv rinses 2x wk  high ph water \nli  if i use one  it s usually cj cali cj h b li  or gdli\nstyling  fsg with avg  an  hemp seed oil   slippery elm  winter only   kccc  cj cccc  cj sg  cj afg  aomm   brhg \n\ncurrently experimenting w dm miracurls cg   mm'

In [63]:
samples[34]

"Mostly 3A, some 3B/2C fine-med texture/med porosity/low density\n\nwash: alternate co-washing with V05 & soap bars\nRinse out: Rotate TJ's N&S, AOHR, LVPNG, GVP K-Pak, CJ CA Daily Condish\nACV rinses 2x/wk (high pH water)\nLI: if I use one, it's usually CJ CALI CJ H&B LI, or GDLI\nStyling: FSG with AVG, AN, hemp seed oil & slippery elm (winter only), KCCC, CJ CCCC, CJ SG, CJ AFG, AOMM & BRHG \n\nCurrently experimenting w/DM Miracurls CG & MM"

In [92]:
# Clean up the signatures
# Take the four characteristics and put them in lists (put absent ones as null values) while removing them
# remove www, http, cg, cgm completly
# put the remainder into a string of no puntation and lowercase(inspect these and maybe apply stop words...maybe)
# Finally read the string 

curls = []
density = []
porosity = []
texture = []

s = samples[34]
s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
s = s.lower()
s = re.sub(r"\n", " ", s)
s = re.sub(r"med", "medium", s)
s = re.sub(r"hi", "high", s)
s = re.sub(r"po", "porosity", s)
s = re.sub(r"den", "density", s)


#curl pattern
curl_pattern = r"\da|\db|\dc"
curl_result1 = re.search(curl_pattern, s)

try:
    s = s.replace(result2.group(), "")
    curls.append(result.group())
except AttributeError:
    #if error put np.nan into list
    curls.append(np.nan)

# Texture Patten
#texture_pattern = r"\w*(?= texture)"


# Porosity Pattern
#texture_pattern = r"\w*(?= porosity)"


# Density Pattern
#texture_pattern = r"\w*(?= density)"

print(s)
print(curls)

mostly 3a  some 3b 2c fine medium texture medium porosityrosity low densitysity  wash  alternate co washighng with v05   soap bars rinse out  rotate tj s n s  aohr  lvpng  gvp k pak  cj ca daily condish acv rinses 2x wk  highgh ph water  li  if i use one  it s usually cj cali cj h b li  or gdli styling  fsg with avg  an  hemp seed oil   slippery elm  winter only   kccc  cj cccc  cj sg  cj afg  aomm   brhg   currently experimenting w dm miracurls cg   mm
[nan]


In [86]:
pat = r"\da|\db|\dc"
pat2=r"astronomy"
result = re.search(pat, s)
result.group()
result2 = re.search(pat2, s)
#s[7:9]

In [89]:
# Use try block to place a null value in that characteristic catagory

try:
    s = s.replace(result2.group(), "")
    curl_pattern.append(result.group())
except AttributeError:
    #if error put np.nan into list
    curl_pattern.append(np.nan)
     
    



[nan]